# 5.Deep Learning Computation.

## 5.1. Layers and Blocks


In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(
    nn.Linear(20,256),
    nn.ReLU(),
    nn.Linear(256,10)
)

X = torch.rand(2, 20)
net(X)

tensor([[ 0.1181, -0.1507, -0.0941,  0.0038, -0.0936, -0.0700,  0.0760, -0.0289,
         -0.0545, -0.0680],
        [ 0.0588, -0.2313, -0.0317, -0.0948, -0.1151, -0.0336,  0.0528, -0.0274,
          0.0200, -0.0653]], grad_fn=<AddmmBackward0>)

### 5.1.1. A Custom Block



In [2]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.hidden = nn.Linear(20,256) # hidden layer
    self.out = nn.Linear(256,10)

  def forward(self, X):
    return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)

tensor([[ 0.2720, -0.3308,  0.1105,  0.0492, -0.0615, -0.1609, -0.0047, -0.0817,
         -0.0504, -0.1572],
        [ 0.2612, -0.3434,  0.0536,  0.0311, -0.1796, -0.1453,  0.0674, -0.0116,
         -0.0268, -0.1635]], grad_fn=<AddmmBackward0>)

### 5.1.2 The Sequential Block

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1587, -0.0718,  0.0641, -0.0423, -0.2909,  0.0091, -0.1087,  0.0209,
          0.0090,  0.1373],
        [-0.1992,  0.0245,  0.1491,  0.1110, -0.2535, -0.0255, -0.0726, -0.0198,
          0.0612,  0.0749]], grad_fn=<AddmmBackward0>)

### 5.1.3. Executing Code in the Forward Propagation Function

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [7]:
net = FixedHiddenMLP()
net(X)

tensor(0.1264, grad_fn=<SumBackward0>)

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 64), 
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
            )
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.2045, grad_fn=<SumBackward0>)

## 5.2. Parameter Management

In [9]:
import torch
from torch import nn

net = nn.Sequential(
    nn.Linear(4,8), 
    nn.ReLU(), 
    nn.Linear(8,1)
    )
X = torch.rand(size=(2,4))
net(X)

tensor([[0.0730],
        [0.1192]], grad_fn=<AddmmBackward0>)

### 5.2.1. Parameter Access

In [10]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.3494, -0.0039, -0.0074,  0.2758,  0.1780,  0.0679,  0.1963, -0.0629]])), ('bias', tensor([0.0058]))])


In [11]:
# 5.2.1.1. Targeted Parameters
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.0058], requires_grad=True)
tensor([0.0058])


In [12]:
net[2].weight.grad == None

True

In [13]:
# 5.2.1.2. All Parameters at Once
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [14]:
net.state_dict()['2.bias'].data

tensor([0.0058])

In [15]:
# 5.2.1.3. Collecting Parameters from Nested Blocks¶
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.0042],
        [-0.0047]], grad_fn=<AddmmBackward0>)

In [16]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [17]:
rgnet[0][1][0].bias.data

tensor([ 0.2019, -0.2769, -0.4830, -0.4563,  0.3786, -0.2827,  0.0457,  0.0085])

### 5.2.2. Parameter Initialization


In [18]:
# 5.2.2.1. Built-in Initialization
def init_normal(m):
  if type(m) == nn.Linear:
    nn.init.normal(m.weight, mean=0, std=0.01)
    nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  after removing the cwd from sys.path.


(tensor([ 0.0045, -0.0030, -0.0132,  0.0018]), tensor(0.))

In [19]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [20]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.0223,  0.3206,  0.1667, -0.2353])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [21]:
# 5.2.2.2. Custom Initialization
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-7.8061,  8.7014, -0.0000,  7.1172],
        [-0.0000,  0.0000, -0.0000,  7.1900]], grad_fn=<SliceBackward0>)

In [22]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  9.7014,  1.0000,  8.1172])

In [23]:
# 5.2.3. Tied Parameters
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


## 5.3. Deferred Initialization


In [24]:
import tensorflow as tf

net = tf.keras.models.Sequential([
  tf.keras.layers.Dense(256, activation = tf.nn.relu),
  tf.keras.layers.Dense(10)
])

In [25]:
[net.layers[i].get_weights() for i in range(len(net.layers))]

[[], []]

In [26]:
X = tf.random.uniform((2,20))
net(X)
[w.shape for w in net.get_weights()]

[(20, 256), (256,), (256, 10), (10,)]

## 5.4.Custom Layers

### 5.4.1. Layers without Parameters

In [27]:
import torch
from torch import nn
from torch.nn import functional as f

class CenteredLayer(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self,X):
    return X - X.mean()

In [28]:
layer = CenteredLayer()
layer(torch.FloatTensor([1,2,3,4,5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [29]:
net = nn.Sequential(
    nn.Linear(8,128),
    CenteredLayer()
)

In [30]:
Y = net(torch.rand(4,8))
Y.mean()

tensor(3.7253e-09, grad_fn=<MeanBackward0>)

### 5.4.2. Layers with Parameters

In [31]:
class MyLinear(nn.Module):
  def __init__(self, in_units, units):
    super().__init__()
    self.weight = nn.Parameter(torch.randn(in_units, units))
    self.bias = nn.Parameter(torch.rand(units, ))

  def forward(self,X):
    linear = torch.matmul(X, self.weight.data) + self.bias.data
    return F.relu(linear)

In [32]:
linear = MyLinear(5,3)
linear.weight

Parameter containing:
tensor([[-0.3171, -1.0824,  0.2915],
        [ 2.4348, -0.1454, -0.8463],
        [ 1.1100, -0.8628,  0.6546],
        [-1.3700,  0.1496,  1.1587],
        [-0.2267, -1.2552,  0.7476]], requires_grad=True)

In [33]:
linear(torch.rand(2, 5))

tensor([[1.3958, 0.0000, 1.4496],
        [1.4827, 0.0000, 0.8372]])

In [34]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])

## 5.5.File I/O

### 5.5.1 Loading and Saving Tensors


In [35]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

In [36]:
x2 = torch.load("x-file")
x2

tensor([0, 1, 2, 3])

In [37]:
y = torch.zeros(4)
torch.save([x,y], 'x-files')
x2,y2 = torch.load("x-files")
(x2,y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [38]:
mydict = {'x':x, 'y':y}
torch.save(mydict, "mydict")
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

### 5.5.2. Loading and Saving Model Parameters


In [39]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [40]:
torch.save(net.state_dict(), 'mlp.params')

In [41]:
clone = MLP()
clone.load_state_dict(torch.load("mlp.params"))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [42]:
Y_clone = clone(X)
Y_clone = Y

## 5.6. GPUs

In [43]:
!nvidia-smi

Sun Feb 13 14:21:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    26W /  70W |   1648MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### 5.6.1. Computing Devices


In [44]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [45]:
torch.cuda.device_count()

1

In [46]:
def try_gpu(i=0): 
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus(): 
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

### 5.6.2. Tensors and GPUs


In [47]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [48]:
# 5.6.2.1. Storage on the GPU
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [49]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.1006, 0.1707, 0.5632],
        [0.0206, 0.7791, 0.1373]])

In [50]:
# 5.6.2.2. Copying
Z = X.cuda(1)
print(X)
print(Z)

RuntimeError: ignored

In [51]:
Y + Z

NameError: ignored

In [52]:
Z.cuda(1) is Z

NameError: ignored

### 5.6.3. Neural Networks and GPUs


In [53]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [54]:
net(X)

tensor([[0.3239],
        [0.3239]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [55]:
net[0].weight.data.device

device(type='cuda', index=0)